In [ ]:
import boto3
import pandas as pd
import pyarrow.parquet as pq
import s3fs
import numpy as np


silver_path = "krish-redditadhd-silver/adhd_posts_silver/"

# Create an S3 filesystem
fs = s3fs.S3FileSystem()

# Load Parquet dataset (automatically reads all partitions)
dataset = pq.ParquetDataset(silver_path, filesystem=fs)

# Convert to Pandas DataFrame
df = dataset.read().to_pandas()
df["clean_title"] = df["clean_title"].fillna("")
df["clean_body"] = df["clean_body"].fillna("")

df["text_for_embedding"] = (
    df["clean_title"].str.strip()
    + "\n\n"
    + df["clean_body"].str.strip()
)

df["text_for_embedding"] = df["text_for_embedding"].replace("", np.nan)
df = df.dropna(subset=["text_for_embedding"])
df.head()


